In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [2]:
model, learning_rate = "BasicNNSplit", .006
# model, learning_rate = "BasicNN", .0054
# model, learning_rate = "LogisticRegressor", .003
epochs = 1000
seed = 1
include_state = True
estate = True
fullq = True
respond_perc = .50
fullseq = False
insertpreds = True
noise = 0.07
smooth = 0

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    modelSplit = True,
    numValFolds = 5,
    epochsToUpdateLabelMods = 10,
    data_kw={"minw": 2,
            "maxw": 31,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq,
             "full_sequence": fullseq,
             "insert_predictions": insertpreds,
             "one_hot_response_features": False,
             "response_feature_noise": noise,
             "max_state_week": 1,
             
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 10,
        "lr_step_mult": .9, 
        "lr_step_epochs": 100,
        "opt_kw": {
            "lr": learning_rate,
        },
        "splitModel": True,
        "labelSmoothPerc": smooth,
        "gaussianNoiseStd": noise
        
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })


print(len(e.bd.test))
print(len(e.bd.train))

report = e.run()






TypeError: append() takes exactly one argument (2 given)

In [ ]:

print (np.mean(report['train_metrics'], axis=0))
labels = report["metric_labels"]
print(report['train_metrics'][-1, labels.index("Acc")])
print(report['test_metrics'][-1, labels.index("Acc")])

splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
plt.title("Train/Test Performance Over Training")
plt.legend()
plt.ylabel("Metric")
plt.xlabel("Training Epoch")
plt.savefig("simpleNotebookAccPlot.png")

plt.clf()

In [ ]:
print(e.bd.features.shape, e.bd.featureList.shape)